$$\huge{\text{Assignment 3 Due 3/4}}$$

<font size =4> __NAME: Nolan Stelzner__
    
 <font size=4>Remember to print the file to pdf and name it:  __Lastname1_Lastname2_Assignment3.pdf__
     

<font color=red>   

 CHECK THE FILE before you submit it to the folder in Teams  



<font size=4>This assignment will be veryclosely related to the __Pyspark Classification Note Book__ we already went over. When looking at the data to check your work you can either use 
    
    df.show() [much faster] or df.topandas().head() [easier to read]


    
We will use data from  __Citi Bike Trip Histories__.  See link for details 
 
https://www.citibikenyc.com/system-data

In [ ]:
!pip install pyspark

<font size=4> Enter the cell below to get started.

In [34]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import * 
import pyspark.sql.functions as F
from pyspark.sql.functions import col, asc,desc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,StandardScaler
from pyspark.ml import Pipeline


spark=SparkSession.builder \
.master ("local[*]")\
.appName("part3")\
.getOrCreate()

In [35]:
sc=spark.sparkContext
sqlContext=SQLContext(sc)

Import file 201307-201402-citibike-tripdata.zip with zipfile and extract all

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
!unzip "/content/drive/My Drive/Pyspark/201307-201402-citibike-tripdata.zip"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
import findspark
findspark.init()

Archive:  /content/drive/My Drive/Pyspark/201307-201402-citibike-tripdata.zip
replace 2014-02 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2014-02 - Citi Bike trip data.csv  
replace 2014-01 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2014-01 - Citi Bike trip data.csv  
replace 2013-12 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2013-12 - Citi Bike trip data.csv  
replace 2013-11 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2013-11 - Citi Bike trip data.csv  
replace 2013-10 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2013-10 - Citi Bike trip data.csv  
replace 2013-09 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2013-09 - Citi Bike trip data.csv  
replace 2013-08 - Citi Bike trip data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2013-08 - Citi Bike trip data.csv 

## <font color =blue> Read File in using spark and name it df. Take a look at it using df.toPandas().head()

In [37]:
df=spark.read \
 .option("header","True")\
 .option("inferSchema","True")\
 .option("sep",",")\
 .csv("2013-09 - Citi Bike trip data.csv")
print("There are",df.count(),"rows",len(df.columns),
      "columns" ,"in the data.") 

There are 1034359 rows 15 columns in the data.


In [ ]:
df.toPandas().head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1010,2013-09-01 00:00:02,2013-09-01 00:16:52,254,W 11 St & 6 Ave,40.735324,-73.998004,147,Greenwich St & Warren St,40.715422,-74.011220,15014,Subscriber,1974,1
1,1443,2013-09-01 00:00:09,2013-09-01 00:24:12,151,Cleveland Pl & Spring St,40.721816,-73.997203,497,E 17 St & Broadway,40.737050,-73.990093,19393,Customer,\N,0
2,1387,2013-09-01 00:00:16,2013-09-01 00:23:23,352,W 56 St & 6 Ave,40.763406,-73.977225,405,Washington St & Gansevoort St,40.739323,-74.008119,16160,Subscriber,1992,1
3,405,2013-09-01 00:00:18,2013-09-01 00:07:03,490,8 Ave & W 33 St,40.751551,-73.993934,459,W 20 St & 11 Ave,40.746745,-74.007756,14997,Subscriber,1973,1
4,270,2013-09-01 00:00:20,2013-09-01 00:04:50,236,St Marks Pl & 2 Ave,40.728419,-73.987140,393,E 5 St & Avenue C,40.722992,-73.979955,19609,Subscriber,1984,1


<font size=4> I'm going to change some of the variable types for you 

In [38]:
from pyspark.sql.types import IntegerType
df =df.withColumn("birth year", df["birth year"].cast(IntegerType()))
df =df.withColumn("tripduration", df["tripduration"].cast(IntegerType()))
df =df.withColumn("bikeid", df["bikeid"].cast(IntegerType()))
df =df.withColumn("start station id", df["start station id"].cast(IntegerType()))
df =df.withColumn("end station id", df["end station id"].cast(IntegerType()))
df =df.withColumn("start station latitude", df["start station latitude"].cast(DoubleType()))
df =df.withColumn("start station longitude", df["start station longitude"].cast(DoubleType()))
df =df.withColumn("end station latitude", df["end station latitude"].cast(DoubleType()))
df =df.withColumn("end station longitude", df["end station longitude"].cast(DoubleType()))


## <font color=blue> Print the Schema

In [ ]:
df.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- start station id: integer (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: integer (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: double (nullable = true)
 |-- end station longitude: double (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: integer (nullable = true)
 |-- gender: integer (nullable = true)



## <font color=blue> Use the PySpark attributes .na.drop and .filter to drop nans and get red of 0's in gender ( unknown gender) 

In [39]:
print("There are",df.count(),"rows",len(df.columns),
      "columns" ,"in the data.") 

There are 1034359 rows 15 columns in the data.


In [40]:
#Dropping all NaN's in data set
df = df.na.drop()

In [41]:
print("There are",df.count(),"rows",len(df.columns),
      "columns" ,"in the data.") 

There are 889190 rows 15 columns in the data.


In [42]:
print("There were",1034359-889190, "NaN's removed from the data set")

There were 145169 NaN's removed from the data set


In [43]:
#Removing all 0's from gender column
df = df.filter("gender != 0").dropna()

In [ ]:
df.toPandas().head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1010,2013-09-01 00:00:02,2013-09-01 00:16:52,254,W 11 St & 6 Ave,40.735324,-73.998004,147,Greenwich St & Warren St,40.715422,-74.011220,15014,Subscriber,1974,1
1,1387,2013-09-01 00:00:16,2013-09-01 00:23:23,352,W 56 St & 6 Ave,40.763406,-73.977225,405,Washington St & Gansevoort St,40.739323,-74.008119,16160,Subscriber,1992,1
2,405,2013-09-01 00:00:18,2013-09-01 00:07:03,490,8 Ave & W 33 St,40.751551,-73.993934,459,W 20 St & 11 Ave,40.746745,-74.007756,14997,Subscriber,1973,1
3,270,2013-09-01 00:00:20,2013-09-01 00:04:50,236,St Marks Pl & 2 Ave,40.728419,-73.987140,393,E 5 St & Avenue C,40.722992,-73.979955,19609,Subscriber,1984,1
4,400,2013-09-01 00:00:35,2013-09-01 00:07:15,490,8 Ave & W 33 St,40.751551,-73.993934,453,W 22 St & 8 Ave,40.744751,-73.999154,17523,Subscriber,1978,1


In [ ]:
df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889130 entries, 0 to 889129
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             889130 non-null  int32  
 1   starttime                889130 non-null  object 
 2   stoptime                 889130 non-null  object 
 3   start station id         889130 non-null  int32  
 4   start station name       889130 non-null  object 
 5   start station latitude   889130 non-null  float64
 6   start station longitude  889130 non-null  float64
 7   end station id           889130 non-null  int32  
 8   end station name         889130 non-null  object 
 9   end station latitude     889130 non-null  float64
 10  end station longitude    889130 non-null  float64
 11  bikeid                   889130 non-null  int32  
 12  usertype                 889130 non-null  object 
 13  birth year               889130 non-null  int32  
 14  gend

## <font color=blue> Let's do a feature creation for the distance between stations using geopy  udf and the lat long coordinates from the data.  Call the new feature station_distance. 
    
<font size=4>The  __Create a new column - Age Groups section__ from the notebook is an example of using udf.
 
Let's start by showing you how geodesic works. you probably need to install geopy first. 

In [44]:
#!pip install geopy #only need this once
from geopy.distance import geodesic
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
geodesic(newport_ri, cleveland_oh).miles


538.3904453677203

In [45]:
from pyspark.sql.functions import lit, struct
def miles_udf(x, y):
    return geodesic(x, y).miles
miles = udf(miles_udf)
start = struct(df['start station latitude'],df['start station longitude'])
stop = struct(df['end station latitude'],df['end station longitude'])
df=df.withColumn("station_distance", miles(start,stop))

In [ ]:
#def udf_multiple(Miles):
#      if (Miles < 100):
#        return 'Under 100 Miles'
#      elif (Miles >= 100 and age <= 250):
#        return 'Between 100 and 250 Miles'
#      elif (Miles > 250 and age < 500):
#        return 'Between 251 and 499 Miles'
#      elif (Miles >= 500):
#        return 'Over 500 Miles'
#      else: return 'N/A'

#latlon_udf = udf(udf_multiple)
#df=df.withColumn("station_distance", latlon_udf('miles'))

## <font color=blue>Now change this new feature into a DoubleType variable.

In [46]:
df =df.withColumn("station_distance", df["station_distance"].cast(DoubleType()))

## <font color =blue>Change the gender data type to Integer so we can do some  correlation.

In [47]:
df =df.withColumn("gender", df["gender"].cast(IntegerType()))

## <font color =blue>Perform a correlation using df.toPandas().corr()

In [ ]:
df.toPandas().corr()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
tripduration,1.000000,0.023017,-0.010405,0.003252,0.017944,-0.017316,0.007305,-0.000238,-0.020667,0.044029
start station id,0.023017,1.000000,0.102007,0.082603,0.040167,0.055533,0.016807,-0.000378,-0.003077,-0.003337
start station latitude,-0.010405,0.102007,1.000000,0.203310,0.046072,0.576201,0.067381,-0.001079,-0.047789,-0.064669
start station longitude,0.003252,0.082603,0.203310,1.000000,0.017083,0.079429,0.403148,-0.003520,0.029406,-0.017939
end station id,0.017944,0.040167,0.046072,0.017083,1.000000,0.095520,0.080092,0.000135,-0.006898,-0.003542
end station latitude,-0.017316,0.055533,0.576201,0.079429,0.095520,1.000000,0.186946,0.000315,-0.053723,-0.062387
end station longitude,0.007305,0.016807,0.067381,0.403148,0.080092,0.186946,1.000000,-0.003105,0.029634,-0.019149
bikeid,-0.000238,-0.000378,-0.001079,-0.003520,0.000135,0.000315,-0.003105,1.000000,-0.001147,-0.000415
birth year,-0.020667,-0.003077,-0.047789,0.029406,-0.006898,-0.053723,0.029634,-0.001147,1.000000,0.064965
gender,0.044029,-0.003337,-0.064669,-0.017939,-0.003542,-0.062387,-0.019149,-0.000415,0.064965,1.000000


<font size=6 color=burntorange> __We are going to try and build a logistic model to try and pick gender from this data.  None of the correlations above are great but le't see how we do.__  

## <font color=blue>  Create a pipeline  to vectorize and standard scale the features and set gender to be the label.  Let's choose the following features for our model.
    
    "tripduration","start station latitude",
                         "start station longitude","end station latitude","end station longitude",
                         "birth year","station_distance"

In [48]:
assembler = VectorAssembler()\
         .setInputCols (["tripduration","start station latitude",
                     "start station longitude","end station latitude","end station longitude",
                     "birth year","station_distance"])\
         .setOutputCol ("vectorized_features")

scaler = StandardScaler()\
         .setInputCol ("vectorized_features")\
         .setOutputCol ("features")
         
label_indexer = StringIndexer()\
         .setInputCol ("gender")\
         .setOutputCol ("label")

pipeline_stages=Pipeline()\
                .setStages([assembler,label_indexer,scaler])
pipeline_model=pipeline_stages.fit(df)
pipeline_df=pipeline_model.transform(df)

In [49]:
pipeline_df.show()

+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+-------------------+--------------------+-----+--------------------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|   station_distance| vectorized_features|label|            features|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+-------------------+--------------------+-----+--------------------+
|        1010|2013-09-01 00:00:

## <font color=blue> Split 80% of the data off for training and run the th model below 
    
    lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=5)
    
<font size=4> Produce a table with only 'label','prediction', 'probability

In [50]:
train, test = pipeline_df.randomSplit([0.8, 0.2], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 711154
Test Dataset Count: 177976


In [53]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=5)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
predictions.select('label','prediction','probability').toPandas().head(5)

,label,prediction,probability
0,0.0,0.0,"[0.7516994588431922, 0.24830054115680777]"
1,0.0,0.0,"[0.7517917524195405, 0.24820824758045945]"
2,0.0,0.0,"[0.7518219945416721, 0.2481780054583279]"
3,0.0,0.0,"[0.7517667750313829, 0.2482332249686171]"
4,1.0,0.0,"[0.7518411720086915, 0.2481588279913085]"


## <font color=blue> Find the accuracy of the model on the test set, the numbers of males in the test set and how many predictions it got correct.

In [55]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count()/float(predictions.count())
print(accuracy)

0.7539387333123567


In [56]:
predictions.groupby('label').count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|134183|
|  1.0| 43793|
+-----+------+



<font size=6 color=burntorange> __Conclusions__

## <font color=blue>Use the results from the cells above to say if you got a good model.  My accuracy was around 75%.  That is almost as good ad the model from the  Pyspark Classification Note Book  we went over in class.

<font color=brown size=4> __Answer: The model accuracy isn't all that great but the worst part about the model is the significant difference between the genders for the predictions.__